# Creational Design Patterns
- [Factory Method](#factory-method)

### Factory Method  [top](#creational-design-patterns)
The factory method works by defining a common interface used by the Creator class and a common interace used by the Product class. In this way the Creator can use objects with the Product type in the same way everytime while the actual identity of the Product itself (the concrete Product) can change depending on the Creator that is instantiated. 

In [1]:
from abc import ABCMeta, abstractmethod 

In [2]:
class AbstractDocument(metaclass=ABCMeta):
    def __init__(self):
        self.data = '' 

    @abstractmethod 
    def open(self): 
        pass 

    @abstractmethod 
    def close(self):
        pass 

    def save(self, path):
        with open(path, 'w') as f: 
            f.write(self.data)
        return None

class AbstractApplication(metaclass=ABCMeta):
    def __init__(self):
        self.data = None
        self.docs = []

    @abstractmethod 
    def create_document(self) -> AbstractDocument:
        pass 

    def new_document(self):
        document = self.create_document()
        self.docs.append(document)
        document.open()


To know when to use the factory method, think like this:  
There is a typical way to perform a function requiring the use of certain products of a certain type.  
The other functions use these products the same way and can be implemented in the abstract class itself but the creation of these products are delegated to a couple of factory methods that decide what kind of products are returned.  
Each concrete Creator specifies the concrete Products that are produced by their factory methods which are then used in the same way by the method inherited from the abstract class. 

If you find the number of factory methods you have to implement increasing, or if you are finding that you need to define more and more concrete products to produce, consider switching to an abstract factory pattern instead. 

### Abstract Factory
If you need to work with various families of related products you can think of using an abstract factory.


In [ ]:
# Imagine that a client wants to build a certain kind of maze and have a player interact with various elements in the maze

In [3]:
from abc import ABCMeta, abstractmethod

class AbstractAttack(metaclass=ABCMeta):
    @property 
    @abstractmethod
    def damage(self):
        pass

class AbstractWall(metaclass=ABCMeta):
    def __init__(self):
        self._hit_points = 200

    @property
    @abstractmethod
    def damage_multiplier(self):
        pass

    def get_hit(self, attack:AbstractAttack):
        self._hit_points - (attack.damage * self.damage_multiplier)
        return None 

class AbstractDoor(metaclass=ABCMeta):
    def __init__(self):
        self._hit_points = 10

    @property 
    @abstractmethod 
    def damage_multiplier(self): 
        pass 
 
    def get_hit(self, attack:AbstractAttack):
        self._hit_points - (attack.damage * self.damage_multiplier)
        return None 

class AbstractFloor(metaclass=ABCMeta):
    @property
    @abstractmethod 
    def speed_multiplier(self):
        pass

class RoomAbstractFactory(metaclass=ABCMeta):
    @abstractmethod
    def create_wall(self) -> AbstractWall: 
        pass 

    @abstractmethod 
    def create_door(self) -> AbstractDoor: 
        pass 

    @abstractmethod 
    def create_floor(self) -> AbstractFloor: 
        pass


In [7]:
class BreakableWall(AbstractWall):
    @property
    def damage_multiplier(self):
        return 1

class UnbreakableWall(AbstractWall):
    @property 
    def damage_multiplier(self):
        return 0

class BreakableDoor(AbstractDoor): 
    @property 
    def damage_multiplier(self):
        return 1

class UnbreakableDoor(AbstractDoor):
    @property 
    def damage_multiplier(self):
        return 0

class StickyFloor(AbstractFloor):
    @property 
    def speed_multiplier(self):
        return 0.5

class SlipperyFloor(AbstractFloor):
    @property 
    def speed_multiplier(self):
        return 1.5

class Floor(AbstractFloor): 
    @property 
    def speed_multiplier(self):
        return 1
    
class StageOneRoomFactory(RoomAbstractFactory):
    def create_door(self):
        return UnbreakableDoor()
    
    def create_floor(self):
        return Floor() 
    
    def create_wall(self):
        return UnbreakableWall()
    

class StageTwoRoomFactory(RoomAbstractFactory):
    def create_door(self):
        return UnbreakableDoor() 
    
    def create_floor(self):
        return StickyFloor()
    
    def create_wall(self):
        return UnbreakableWall() 
    

class StageThreeRoomFactory(RoomAbstractFactory):
    def create_door(self):
        return BreakableDoor() 
    
    def create_floor(self):
        return SlipperyFloor() 
    
    def create_wall(self):
        return BreakableWall()

In [17]:
class Game:
    def __init__(self):
        self._character_x_pos = 0
        self._character_y_pos = 0

    def build_stage(self, room_factory:RoomAbstractFactory):
        self._character_x_pos = 0
        self._character_y_pos = 0
        self.door = room_factory.create_door()
        self.floor = room_factory.create_floor() 
        self.wall = room_factory.create_wall() 

    @property
    def character_pos(self):
        return self._character_x_pos, self._character_y_pos 
    
    def move_character(self, x, y):
        print("moving from", self.character_pos, end=' ')
        self._character_x_pos = self._character_x_pos + (x * self.floor.speed_multiplier)
        self._character_y_pos = self._character_y_pos + (y * self.floor.speed_multiplier)
        print("to", self.character_pos)

In [22]:
game = Game() 
game.build_stage(StageOneRoomFactory())
game.move_character(1, 2)
game.move_character(2, 10)
game.build_stage(StageTwoRoomFactory())
game.move_character(5, 10)
game.move_character(10, 7)
game.build_stage(StageThreeRoomFactory())
game.move_character(4, 2)

moving from (0, 0) to (1, 2)
moving from (1, 2) to (3, 12)
moving from (0, 0) to (2.5, 5.0)
moving from (2.5, 5.0) to (7.5, 8.5)
moving from (0, 0) to (6.0, 3.0)


Now, say you wish to build a Stage 4, there is already a fixed way to do so using the Abstract Factory Pattern. And if you wish to implement a new kind of room component, you can use extend the respective Product interfaces.

In [23]:
class AcceleratorFloor(AbstractFloor):
    @property
    def speed_multiplier(self):
        return 2

class StageFourRoomFactory(RoomAbstractFactory):
    def create_door(self):
        return BreakableDoor()
    
    def create_floor(self):
        return AcceleratorFloor()
    
    def create_wall(self):
        return BreakableWall() 

In [24]:
game.build_stage(StageFourRoomFactory())
game.move_character(4, 4)

moving from (0, 0) to (8, 8)


The obvious benefit here is that the Client Code doesn't have to implement their factory methods on their end, they just have to pass in the appropriate concrete factory at the appropriate time.  
It also gets to keep the item creation logic within a separate object, closer to the Single Responsibility Principle

Let's go back to the very beginning and follow along with the example provided by the book Design Patterns

In [ ]:
from abc import ABCMeta, abstractmethod 

class MapSite(metaclass=ABCMeta):
    @abstractmethod 
    def enter(self):
        pass 

class Room(MapSite):
    def __init__(self, room_no): 
        pass 
    
    def get_side(direction):
        pass 

    def set_side(direction, map_site:MapSite): 
        pass 

